In [1]:
#!git clone https://github.com/Aniruddha-Ponnuri/Amazon--ML-challenge.git

In [2]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import re
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import sys
#sys.path.append('/content/Amazon--ML-challenge/')
from src.utils import parse_string
from src.constants import entity_unit_map
from src.sanity import sanity_check
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 1. Load Data
train_df = pd.read_csv('dataset/sample.csv')
test_df = pd.read_csv('dataset/sample_test.csv')

In [4]:
# 2. Process Images Directly from URLs
def load_and_preprocess_image_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        img = img.resize((224, 224))
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array
    except Exception as e:
        print(f"Error loading image from {url}: {e}")
        # Create a black placeholder image
        img = Image.new('RGB', (224, 224), color='black')
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array

In [5]:
# 3. Extract Features Using ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [6]:
# 4. Prepare Labels for Training
def extract_value_and_unit(entity_value):
    pattern = r'^(\d+\.?\d*)\s+([a-zA-Z\s]+)$'
    match = re.match(pattern, str(entity_value).strip())
    if match:
        value = float(match.group(1))
        unit = match.group(2).strip().lower()
        return value, unit
    else:
        return None, None

In [7]:
train_df['value_unit'] = train_df['entity_value'].apply(extract_value_and_unit)
train_df = train_df.dropna(subset=['value_unit'])
train_df['value'] = train_df['value_unit'].apply(lambda x: x[0])
train_df['unit'] = train_df['value_unit'].apply(lambda x: x[1])
train_df = train_df.drop(columns=['value_unit'])

In [8]:
train_df.head()

,image_link,group_id,entity_name,entity_value,value,unit
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,500.000,gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,1.000,cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,0.709,gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,0.709,gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,1400.000,milligram


In [9]:
# Encode units per entity_name
from collections import defaultdict

unit_encoders = defaultdict(LabelEncoder)

def encode_unit(row):
    entity = row['entity_name']
    unit = row['unit']
    le = unit_encoders[entity]
    # Check if the LabelEncoder has been fitted
    if not hasattr(le, 'classes_') or len(le.classes_) == 0:
        le.fit(train_df[train_df['entity_name'] == entity]['unit'].unique())
    return le.transform([unit])[0]


train_df['unit_encoded'] = train_df.apply(encode_unit, axis=1)

In [10]:
train_df.head()

,image_link,group_id,entity_name,entity_value,value,unit,unit_encoded
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,500.000,gram,0
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,1.000,cup,1
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,0.709,gram,0
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,0.709,gram,0
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,1400.000,milligram,2


In [11]:
# 5. Extract Features for Training Images
print("Extracting features from training images...")
train_features = []
train_values = []
train_units = []
for idx, row in tqdm(train_df.iterrows(), total=train_df.shape[0]):
    image_link = row['image_link']
    entity_name = row['entity_name']
    img_array = load_and_preprocess_image_from_url(image_link)
    features = base_model.predict(img_array)
    features = features.reshape(-1)
    train_features.append(features)
    train_values.append(row['value'])
    train_units.append(row['unit_encoded'])

X_train = np.array(train_features)
y_value = np.array(train_values).astype('float32')
y_unit = np.array(train_units).astype('int32')

Extracting features from training images...


  0%|          | 0/199 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  1%|          | 1/199 [00:01<04:57,  1.50s/it]

1/1 [==============================] - 0s 85ms/step


...

 99%|█████████▉| 197/199 [01:02<00:00,  3.29it/s]

1/1 [==============================] - 0s 80ms/step


 99%|█████████▉| 198/199 [01:03<00:00,  3.11it/s]

1/1 [==============================] - 0s 95ms/step


100%|██████████| 199/199 [01:03<00:00,  3.13it/s]


In [12]:
# 6. Build and Train the Model
input_shape = X_train.shape[1]
inputs = Input(shape=(input_shape,))

x = Dense(1024, activation='relu')(inputs)
x = Dropout(0.5)(x)

value_output = Dense(1, name='value_output')(x)

max_unit_classes = max([len(le.classes_) for le in unit_encoders.values()])
unit_output = Dense(max_unit_classes, activation='softmax', name='unit_output')(x)

model = Model(inputs=inputs, outputs=[value_output, unit_output])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss={'value_output': 'mean_squared_error', 'unit_output': 'sparse_categorical_crossentropy'},
              loss_weights={'value_output': 1.0, 'unit_output': 1.0})

# Split data
X_train_split, X_val_split, y_value_train, y_value_val, y_unit_train, y_unit_val = train_test_split(
    X_train, y_value, y_unit, test_size=0.2, random_state=42)

# Train the model
epochs = 10
batch_size = 32

print("Training the model...")
history = model.fit(X_train_split, {'value_output': y_value_train, 'unit_output': y_unit_train},
                    validation_data=(X_val_split, {'value_output': y_value_val, 'unit_output': y_unit_val}),
                    epochs=epochs, batch_size=batch_size)

Training the model...
Epoch 1/10
5/5 [==============================] - 3s 429ms/step - loss: nan - value_output_loss: nan - unit_output_loss: 2.4128 - val_loss: nan - val_value_output_loss: nan - val_unit_output_loss: 1.9455
Epoch 2/10
5/5 [==============================] - 2s 362ms/step - loss: nan - value_output_loss: nan - unit_output_loss: 1.9453 - val_loss: nan - val_value_output_loss: nan - val_unit_output_loss: 1.9451
Epoch 3/10
5/5 [==============================] - 2s 365ms/step - loss: nan - value_output_loss: nan - unit_output_loss: 1.9449 - val_loss: nan - val_value_output_loss: nan - val_unit_output_loss: 1.9447
Epoch 4/10
5/5 [==============================] - 2s 357ms/step - loss: nan - value_output_loss: nan - unit_output_loss: 1.9444 - val_loss: nan - val_value_output_loss: nan - val_unit_output_loss: 1.9443
Epoch 5/10
5/5 [==============================] - 2s 362ms/step - loss: nan - value_output_loss: nan - unit_output_loss: 1.9440 - val_loss: nan - val_value_output

In [13]:
test_df = pd.read_csv('dataset/test.csv')

In [14]:
# 7. Predict Entity Values for test.csv
print("Extracting features from test images...")
test_features = []
entity_names = []
for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    image_link = row['image_link']
    entity_name = row['entity_name']
    entity_names.append(entity_name)
    img_array = load_and_preprocess_image_from_url(image_link)
    features = base_model.predict(img_array)
    features = features.reshape(-1)
    test_features.append(features)

X_test = np.array(test_features)

Extracting features from test images...


  0%|          | 0/131187 [00:00<?, ?it/s]

1/1 [==============================] - 0s 83ms/step


  0%|          | 1/131187 [00:00<7:00:26,  5.20it/s]

1/1 [==============================] - 0s 93ms/step


...

  9%|▉         | 12291/131187 [58:12<9:50:57,  3.35it/s]

1/1 [==============================] - 0s 114ms/step


  9%|▉         | 12292/131187 [58:12<10:05:18,  3.27it/s]

1/1 [==============================] - 0s 116ms/step


  9%|▉         | 12293/131187 [58:13<10:25:50,  3.17it/s]

In [ ]:
# Make predictions
print("Making predictions on test data...")
pred_values, pred_units_probs = model.predict(X_test)

# 8. Format Predictions According to Challenge Rules
# Map predicted units back to actual unit labels
predicted_units = []
for i, entity_name in enumerate(entity_names):
    le = unit_encoders.get(entity_name)
    if le is not None:
        unit_probs = pred_units_probs[i]
        num_classes = len(le.classes_)
        unit_index = np.argmax(unit_probs[:num_classes])
        unit = le.inverse_transform([unit_index])[0]
    else:
        unit = ''
    predicted_units.append(unit)

In [ ]:
# Combine predicted values and units
pred_values_formatted = [str(round(val[0], 2)) for val in pred_values]

predictions = []
for value, unit, entity_name in zip(pred_values_formatted, predicted_units, entity_names):
    allowed_units = entity_unit_map[entity_name]
    if unit in allowed_units:
        prediction = f"{value} {unit}"
        try:
            parse_string(prediction)
            predictions.append(prediction)
        except ValueError:
            predictions.append('')
    else:
        predictions.append('')

In [ ]:
# 9. Save Predictions to CSV
output_df = pd.DataFrame({
    'index': test_df['index'],
    'prediction': predictions
})

assert len(output_df) == len(test_df), "Mismatch in number of predictions."

output_df.to_csv('test_out.csv', index=False)
print("Predictions saved to test_out.csv")


In [ ]:
# 10. Run Sanity Checker
print("Running sanity checker...")
try:
    sanity_check('dataset/test.csv', 'test_out.csv')
    print("Sanity check passed.")
except Exception as e:
    print(f"Sanity check failed: {e}")